In [1]:
from itertools import groupby
from operator import itemgetter

In [2]:
# Now we can proceed to write a comparrison function to parse these exon-intron chains
# the function is not expected to be particularly fast
# however, what I need is just to get an answer, so...

def checkCompat(orf,exons):
    # first convert the orf into lists
    tmp=orf.split("-")
    orfStarts=[int(x) for x in tmp[0].split(",")]
    orfEnds=[int(x) for x in tmp[-1].split(",")]
    
    # next we need to parse the exons in the same manner
    tmp=exons.split("-")
    exonStarts=[int(x) for x in tmp[0].split(",")]
    exonEnds=[int(x) for x in tmp[-1].split(",")]
    numExons=len(exonStarts)
    
    # now we need to compare them to one another
    # there could be three possibilities which we shall use the following return codes for:
    # 0 - ORF fits entirely within the exon
    # 1 - section of the ORF is lost without a frameshift
    # 2 - an extra section is added to the ORF without causing a framesift
    # 3 - there is a frameshift due to an extra section added to the ORF
    # 4 - there is a frameshift due to a section removed from the ORF
    # 5 - ORF is trimmed at the end - perhaps the exon should be extended???
    # 6 - potentially missing start codon - if the beginning of the ORF is trimmed
    
    assert len(orfStarts)==len(orfEnds),"wrong CDS chain passed, length don't match: "+orf
    assert len(exonStarts)==len(exonEnds),"wrong exon chain passed, length don't match: "+exons
#     assert len(orfStarts)<=len(exonStarts),"There are more segments in ORF than in the exon chain. ORF: "+orf+" ; Exons-chain: "+exons
    
    frameShift=0
    tmpFrameShift=0 # used when novel introns are present within a known ORF - multiple frameShifts need to be evaluated
    modification=0
    
    orfPos=0
    exonPos=-1
    numOrfSegments=len(orfStarts)
    for es,ee in zip(exonStarts,exonEnds):
        exonPos+=1
        print("==========",es)
        curOrfStart=orfStarts[orfPos]
        curOrfEnd=orfEnds[orfPos]
        
        # if this happens before the first ORF - then good
        # otherwise we have an insertion and need to check for a frameshift
        if ee<curOrfStart:
            if orfPos==0:
                print("continue 0")
                continue
            else:
                print("inserted exon")
                frameShift=((ee+1)-es)%3
                if frameShift==0:
                    modification=2
                else:
                    return 3
        
        # ORF within exon and fits nicely - exon/CDS segment share end
        elif es<=curOrfStart and ee==curOrfEnd:
            print("should 1")
            if orfPos==numOrfSegments-1: # ORF fits and is done
                print("end 1")
                return modification
            else: # ORF continues
                if exonPos==numExons-1: # trimmed based on having a shorter exon chain
                    return 5
                print("continue 1")
                orfPos+=1
        
        # could be the proper end of the ORF or a signal for modification
        elif es<=curOrfStart and ee>curOrfEnd:
            print("should 2")
            if orfPos==numOrfSegments-1: # ORF fits and is done
                print("end 2")
                return modification
            else: # ORF does not match current exon chain
                print("weird 2")
        
        # could be the end of the ORF, or a signal for modification
        elif es<=curOrfStart and ee<curOrfEnd:
            print("should 3")
            # this is another perplexing case
            # if an exon exists such that it ends before the end of the current ORF segment
            # but there is another exon which fits within the current ORF segment
            # then there is a possibility that two frameshifts will cancel out one another
            frameShift=(curOrfEnd-ee)%3
            if frameShift==0:
                print("frameshift=0 3")
                modification=1
                continue
            else:
                if exonPos<numExons-1 and exonStarts[exonPos+1]<curOrfEnd: # additional frameshifts need to be evaluated
                    print("writing tmp frameshift")
                    tmpFrameShift=frameShift
                    continue
                else:
                    return 4
        
        # frameshift due to exon beginning later
        elif es>curOrfStart and ee==curOrfEnd:
            print("should 4")
            frameShift=((es-curOrfStart)+tmpFrameShift)%3
            if orfPos==numOrfSegments-1:
                print("end 4")
                if frameShift==0:
                    return 1
                else:
                    return 4
            else:
                print("weird 4")
                if frameShift==0:
                    orfPos+=1
                    modification=1
                    continue
                else:
                    return 4
        
        # Something
        elif es>curOrfStart and ee>curOrfEnd:
            print("should 5")
            frameShift=(es-curOrfStart)%3
            if orfPos==numOrfSegments-1:
                print("end 5")
                if frameShift==0:
                    return 1
                else:
                    return 4
            else:
                if frameShift==0:
                    print("weird 5")
                else:
                    return 4 # just abandon - not going to be any worth - it's already a frameshift
        
        # an intron appeared in the middle of the ORF
        elif es>curOrfStart and ee<curOrfEnd:
            print("should 6")
            # this is another perplexing case
            # if an exon exists such that it ends before the end of the current ORF segment
            # but there is another exon which fits within the current ORF segment
            # then there is a possibility that two frameshifts will cancel out one another

            # here frameshift needs to take into account the previously seen tmpFrameShift
            print(tmpFrameShift)
            frameShift=((curOrfEnd-ee)+((es-curOrfStart)+tmpFrameShift))%3
            tmpFrameShift=frameShift
            print(curOrfStart,curOrfEnd,es,ee,tmpFrameShift)
            if frameShift==0:
                print("frameshift=0 6")
                modification=1
                continue
            else:
                if exonPos<numExons-1 and exonStarts[exonPos+1]<curOrfEnd: # additional frameshifts need to be evaluated
                    print("writing tmp frameshift 6")
                    tmpFrameShift=frameShift
                    continue
                else:
                    return 4
            
        else:
            print("ELSE:",es,ee,curOrfStart,curOrfEnd)
            break
                
    return modification
    
knownExons="1,200-100,300"
knownCDS="50,200-100,250"
novelExons0="1,200,400-100,350,500" # 0 - the known ORF fits nicely
novelExons1="1,203-100,300" # 1 - the known ORF does not fit entirely. However the deletion does not cause a frameshift
novelExons2="1,150,200-100,161,300" # 2 - the known ORF does not fit entirely. However, because it is 12 bases that are added - there is no frame shift
novelExons3="1,150,200-100,162,300" # 3 - the known ORF does not fit entirely and the insertion causes a frameshift
novelExons4="1,210-100,300" # 4 - the known ORF does not fit entirely and the deletion causes a frameshift
novelExons5="1,200-100,240"
novelExons6="60,200-100,300"
novelExons7="1-100" # first segment of the exon chain fits well, but the remaining segments are trimmed


# let's now deal with the second batch of test cases
novelExons1="1,200-97,300"

# test for the case when an extra intron splits an ORF segment in two
novelExons1="1,78,200-71,100,300"

# to make things even more complicated - what if we introduce two novel introns into this story?
# the algorithm should be able to handle them easily
knownCDS="1,200-100,250"
novelExons1="1,68,78,200-63,71,100,300"

# same as before, but a simplified example for ease of manual verification
# knownCDS="1-15"
# novelExons1="1,6,13-3,11,15"

# yet another case when a series of introns interrupt + trimming of the end
# novelExons1="1,63,78-68,71,99"

# needs to be able to reset the tmpFrameShift

# print(checkCompat(knownCDS,knownExons))
# print("\n+++++++++++++++\n")
# print(checkCompat(knownCDS,novelExons0))
# print("\n+++++++++++++++\n")
print(checkCompat(knownCDS,novelExons1))
# print("\n+++++++++++++++\n")
# print(checkCompat(knownCDS,novelExons2))
# print("\n+++++++++++++++\n")
# print(checkCompat(knownCDS,novelExons3))
# print("\n+++++++++++++++\n")
# print(checkCompat(knownCDS,novelExons4))
# print("\n+++++++++++++++\n")
# print(checkCompat(knownCDS,novelExons5))
# print("\n+++++++++++++++\n")
# print(checkCompat(knownCDS,novelExons6))
# print("\n+++++++++++++++\n")
# print(checkCompat(knownCDS,novelExons7))

# needs to also account for trimming, when the ORF has more segments than the exon-chain to which it is compared

# So far we've tested cases when it's the exonStart that affects the match
# now need to test the reverse - exonEnd doesn't fit
# tests should be performed for each of the 6 cases as well

# next we also need to test for the presence of multiple modifications and whether they are reported as needed

# also need to test longer exon chains

# also need to test longer CDS chains

========== 1
should 3
writing tmp frameshift
========== 68
should 6
1
1 100 68 71 1
writing tmp frameshift 6
========== 78
should 4
weird 4
========== 200
should 2
end 2
1


In [3]:
def checkCompat(orf,exons):
    # first convert the orf into lists
    tmp=orf.split("-")
    orfStarts=[int(x) for x in tmp[0].split(",")]
    orfEnds=[int(x) for x in tmp[-1].split(",")]
    
    # next we need to parse the exons in the same manner
    tmp=exons.split("-")
    exonStarts=[int(x) for x in tmp[0].split(",")]
    exonEnds=[int(x) for x in tmp[-1].split(",")]
    
    numExons=len(exonStarts)-1
    numOrfSegments=len(orfStarts)-1
    
    assert len(orfStarts)==len(orfEnds),"wrong CDS chain passed, length don't match: "+orf
    assert len(exonStarts)==len(exonEnds),"wrong exon chain passed, length don't match: "+exons
    
    orfPos=0
    exonPos=0
    
    code=""
    orfNotStarted=True # orf has not been reached yet
    
    while True:
        es=exonStarts[exonPos]
        ee=exonEnds[exonPos]
        os=orfStarts[orfPos]
        oe=orfEnds[orfPos]
        # skip to the exon that overlaps the start of the ORF if such exists
        if ee<os:
            if exonPos==numExons and orfNotStarted:
                return "9999999"
            exonPos+=1
        else:
            print("es",es,ee,os,oe)
            if es>=os:
                print("if 1")
                if ee==oe:
                    print("if 2")
                    if orfPos==numOrfSegments:
                        return code+";0"
                    exonPos+=1
                    orfPos+=1
                    continue
                elif ee>oe:
                    print("elif 2")
                    if orfPos==numOrfSegments:
                        return code
                    else: # exon extends past the end of the orf segment - orf3-chain1
                        # deal with the frameshift here
                        
                        # if exon fills the entire ORF intron -> os(next orf)-oe(current orf) -> change exon start to the end of orf intron (orf3-chain1)
                        if ee>orfStarts[orfPos+1]:
                            print("if 3")
                            numAdded=(orfStarts[orfPos+1]-1)-oe
                            frameshift=numAdded%3
                            if frameshift==0:
                                code+=";2"
                            else:
                                code+=";3"
                            
                            exonStarts[exonPos]=orfStarts[orfPos+1]
                            orfPos+=1
                        # else if exon fills part of the ORF intron -> ee-oe -> (orf3-chain2)
                        else:
                            print("else 3")
                            numAdded=ee-oe
                            frameshift=numAdded%3
                            if frameshift==0:
                                code+=";2"
                            else:
                                code+=";3"
                            
                            exonPos+=1
                            orfPos+=1
                else:
                    print("else 4")
                    if exonPos==numExons:
                        return code+";5"
                    else:
                        print("else 5")
                        # if orf fills the entire space between two exons
                        if oe>exonStarts[exonPos+1]:
                            print("if 6")
                            numRemoved=(exonStarts[exonPos+1]-1)-ee
                            frameshift=numRemoved%3
                            if frameshift==0:
                                code+=";1"
                            else:
                                code+=";4"
                            
                            orfStarts[orfPos]=exonStarts[exonPos]
                            exonPos+=1
                        else:
                            print("else 6")
                            numRemoved=oe-ee
                            frameshift=numremoved%3
                            if frameshift==0:
                                code+=";1"
                            else:
                                code+=";4"
                            
                            exonPos+=1
                            orfPos+=1

            else:
                return "adsjflkads"
                

In [160]:
# from https://codereview.stackexchange.com/questions/178427/given-2-disjoint-sets-of-intervals-find-the-intersections

def getIntersection(interval_1, interval_2):
    start = max(interval_1[0], interval_2[0])
    end = min(interval_1[1], interval_2[1])
    if start <= end:
        return (start, end)
    return None

def getIntersection2(interval_1,interval_2):
    left_start=

def return_intersections(intervals1, intervals2):
    iter1 = iter(intervals1)
    iter2 = iter(intervals2)

    interval1 = next(iter1)
    interval2 = next(iter2)

    while True:
        intersection = getIntersection(interval1, interval2)
        if intersection:
            yield intersection
            try:
                if intersection[1] == interval1[1]:
                    interval1 = next(iter1)
                else:
                    interval2 = next(iter2)
            except StopIteration:
                return
        
        else: # need to handle this last case so it does not go into an infinite loop
            return
        
        try:
            while interval1[0] > interval2[1]:
                interval2 = next(iter2)
            while interval2[0] > interval1[1]:
                interval1 = next(iter1)
        except StopIteration:
            return
        
orf1=[(1,18)]
orf3=[(1,3),(7,15)]
chain1=[(1,18)]
chain2=[(1,6),(10,18)]
chain17=[(1,2),(8,9),(15,18)]
list(return_intersections(orf3,chain17))

[(1, 2), (8, 9), (15, 15)]

In [4]:
# what if we try sets
# resChain=[]
# for t in chain2:
#     resChain=resChain+list(range(t[0],t[1]))
# resOrf=[]
# for t in orf3:
#     resOrf=resOrf+list(range(t[0],t[1]))
    
# print(set(resOrf)-set(resChain))
# print(set(resChain)-set(resOrf))

def checkCompat(orf,chain):
    tmp=orf.split("-")
    orfStarts=[int(x) for x in tmp[0].split(",")]
    orfEnds=[int(x) for x in tmp[-1].split(",")]
    
    # next we need to parse the exons in the same manner
    tmp=chain.split("-")
    chainStarts=[int(x) for x in tmp[0].split(",")]
    chainEnds=[int(x) for x in tmp[-1].split(",")]
    
    orf=[]
    for t in zip(orfStarts,orfEnds):
        orf+=list(range(t[0],t[1]+1))
    
    chain=[]
    for t in zip(chainStarts,chainEnds):
        chain+=list(range(t[0],t[1]+1))
    
    # now remove anything in the chain that is before or after the start and end of the ORF
    chain=[x for x in chain if x>=orf[0] and x<=orf[-1]]
    
    orf=set(orf)
    print(orf)
    chain=set(chain)
    print(chain)
    
    # now get set differences
    orf_chain=orf-chain
    chain_orf=chain-orf
    print(orf_chain)
    print(chain_orf)
    
    if len(orf_chain)==0 and len(chain_orf)==0:
        print("hooray")
        return "0"
    
    allMis=[]
    for k, g in groupby(enumerate(orf_chain), lambda ix : ix[0] - ix[1]):
        allMis.append(list(map(itemgetter(1), g)))
    for k, g in groupby(enumerate(chain_orf), lambda ix : ix[0] - ix[1]):
        allMis.append(list(map(itemgetter(1), g)))
        
    print(allMis)
    # now just need to figure out what kind of frameshifts we've got here
    offFrame=0 # number of extra or missing bases that do result in a frameshift
    onFrame=0 # number of extra or missing bases that do not result in a frameshift
    
    # for now this will be a very easy thing to do
    # would be better to have a method for computing how many bases are in off-frame vs on-frame regions
    for mis in allMis:
        if not len(mis)%3==0:
            return 2
    return 1
    
orf3="1,7-3,15"
chain17="1,9,15-2,9,18"
checkCompat(orf3,chain17)

# need to figure out a way to replace the codes with percentage as follows:
# (number of bases in off-frame)/(total number of bases in the ORF)

{1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15}
{1, 2, 9, 15}
{3, 7, 8, 10, 11, 12, 13, 14}
set()
[[3], [7, 8], [10, 11, 12, 13, 14]]


2

In [8]:
# what if we try sets
# resChain=[]
# for t in chain2:
#     resChain=resChain+list(range(t[0],t[1]))
# resOrf=[]
# for t in orf3:
#     resOrf=resOrf+list(range(t[0],t[1]))
    
# print(set(resOrf)-set(resChain))
# print(set(resChain)-set(resOrf))

def checkCompat(orf,chain):
    tmp=orf.split("-")
    orfStarts=[int(x) for x in tmp[0].split(",")]
    orfEnds=[int(x) for x in tmp[-1].split(",")]
    
    # next we need to parse the exons in the same manner
    tmp=chain.split("-")
    chainStarts=[int(x) for x in tmp[0].split(",")]
    chainEnds=[int(x) for x in tmp[-1].split(",")]
    
    orf=[]
    for t in zip(orfStarts,orfEnds):
        orf+=list(range(t[0],t[1]+1))
    
    chain=[]
    for t in zip(chainStarts,chainEnds):
        chain+=list(range(t[0],t[1]+1))
    
    # now remove anything in the chain that is before or after the start and end of the ORF
    chain=[x for x in chain if x>=orf[0] and x<=orf[-1]]
    
    orf=set(orf)
    print(orf)
    chain=set(chain)
    print(chain)
    
    # now get set differences
    orf_chain=orf-chain
    chain_orf=chain-orf
    print(orf_chain)
    print(chain_orf)
    
    if len(orf_chain)==0 and len(chain_orf)==0:
        print("hooray")
        return "0"
    
    allMis=[]
    allMis_orf_chain=[]
    allMis_chain_orf=[]
    for k, g in groupby(enumerate(orf_chain), lambda ix : ix[0] - ix[1]):
        allMis_orf_chain.append(list(map(itemgetter(1), g)))
        allMis.append(list(map(itemgetter(1),g)))
    for k, g in groupby(enumerate(chain_orf), lambda ix : ix[0] - ix[1]):
        allMis_chain_orf.append(list(map(itemgetter(1), g)))
        allMis.append(list(map(itemgetter(1),g)))
        
    print(allMis_orf_chain)
    print(allMis_chain_orf)
    print(allMis)
    # now just need to figure out what kind of frameshifts we've got here
    
    # let's try to make it better and compute percentage of off-frame bases instead
    
    # one of the first differences should be:
    # we need to compute what's missing from the ORF
    # and what is added to the ORF separately
    # meaning two different lists - one as orf.difference(chain)
    # and the other as chain.difference(orf)
    
    # If a frameshift does happen - we want to know how many bases of the ORF are affected
    # basically, what we need to do instead of computing frameshifts
    # we want to know how many bases are different due to the given modifications due to the peptide sequences
    
    # one thing to take into account is that the two sets are guaranteed to be non-overlapping
    # we shall take advantage of that, since we know we can order them
    
    # first thing first - figure out whether the ORF start is ocvered by the exon:    
    if not next(iter(orf))==next(iter(chain)): # if exon does not cover the start codon
        return 555555
    
    offFrame=0 # number of extra or missing bases that do result in a frameshift
    onFrame=0 # number of extra or missing bases that do not result in a frameshift
    
    removed=False # whether the elements in the current array have been removed or added to the original ORF
    curChain=[]
    while True: # parsing through all modifications
        # first figure out which one should be evaluated now: allMis_orf_chain or the allMis_chain_orf
        if not allMis_orf_chain and not allMis_chain_orf:
            return
        elif not allMis_orf_chain: # allMis_orf_chain is empty
            curChain=allMis_chain_orf.pop(0)
            removed=False
        elif not allMis_chain_orf:# allMis_chain_orf is empty
            curChain=allMis_orf_chain.pop(0)
            removed=True
        elif allMis_chain_orf[0]<allMis_orf_chain[0]: # present in the new transcript but not in the orf
            curChain=allMis_chain_orf.pop(0)
            removed=False
        else: # present in the ORF but not the exon chain
            curChain=allMis_orf_chain.pop(0)
            removed=True
        
        # now to proceed with the current chain
        # in order to evaluate what overlapping sections of the ORF are being 
        
orf3="1,7-3,15"
chain17="1,9,15-2,9,18"
checkCompat(orf3,chain17)

# need to figure out a way to replace the codes with percentage as follows:
# (number of bases in off-frame)/(total number of bases in the ORF)

{1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15}
{1, 2, 9, 15}
{3, 7, 8, 10, 11, 12, 13, 14}
set()
[[3], [7, 8], [10, 11, 12, 13, 14]]
[]
[[], [], []]
{1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15}
{1, 2, 9, 15}


In [112]:
# orf3:    1+  +  +3           7+  +  +   +  +  +   +  +  +15
# orf2:              4+  +  +   +  +  +   +  +  +   +  +  +15
# orf1:    1+  +  +   +  +  +   +  +  +   +  +  +   +  +  +   +  +  +18
# exons:   |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |
# pos:      1  2  3   4  5  6   7  8  9   10 11 12  13 14 15  16 17 18
# chain1:  1=  =  =   =  =  =   =  =  =   =  =  =   =  =  =   =  =  =18
# chain2:  1=  =  =   =  =  =6          10=  =  =   =  =  =   =  =  =18 # compatible - deleted ORF segment

orf3="1,7-3,15"
orf2="4-15"
orf1="1-18"

chain1="1-18"
chain2="1,10-6,18"

# print("0 :",checkCompat(orf1,chain1))
# print("1 :",checkCompat(orf1,chain2))
# print("0 :",checkCompat(orf2,chain1))
# print("1 :",checkCompat(orf2,chain2))
# print("2 :",checkCompat(orf3,chain1))
print("2;1 :",checkCompat(orf3,chain2))

es 1 6 1 3
if 1
elif 2
else 3
es 10 18 7 15
if 1
elif 2
2;1 : ;2


In [121]:
# orf3:    1+  +  +3           7+  +  +   +  +  +   +  +  +15
# orf2:              4+  +  +   +  +  +   +  +  +   +  +  +15
# orf1:    1+  +  +   +  +  +   +  +  +   +  +  +   +  +  +   +  +  +18
# exons:   |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |
# pos:      1  2  3   4  5  6   7  8  9   10 11 12  13 14 15  16 17 18
# chain3:  1=  =  =   =  =5             10=  =  =   =  =  =   =  =  =18 # frameshift(1) - deleted ORF segment
# chain4:  1=  =  =   =  =5                11=  =   =  =  =   =  =  =18 # frameshift(2) - deleted ORF segment
# chain5:  1=  =  =   =  =5                   12=   =  =  =   =  =  =18 # compatible - deleted ORF segment

orf3="1,7-3,15"
orf2="4-15"
orf1="1-18"

chain3="1,10-5,18"
chain4="1,11-5,18"
chain5="1,12-5,18"

# orf3:    1+  +  +3           7+  +  +   +  +  +   +  +  +15
# orf2:              4+  +  +   +  +  +   +  +  +   +  +  +15
# orf1:    1+  +  +   +  +  +   +  +  +   +  +  +   +  +  +   +  +  +18
# exons:   |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |
# pos:      1  2  3   4  5  6   7  8  9   10 11 12  13 14 15  16 17 18
# chain6:  1=  =  =   =  =  =6             11=  =   =  =  =   =  =  =18 # frameshift(1) - deleted ORF segment
# chain7:  1=  =  =   =  =5                11=  =   =  =  =   =  =  =18 # frameshift(2) - deleted ORF segment
# chain8:  1=  =  =   =4                   11=  =   =  =  =   =  =  =18 # compatible - deleted ORF segment

orf3="1,7-3,15"
orf2="4-15"
orf1="1-18"

chain6="1,11-6,18"
chain7="1,11-5,18"
chain8="1,11-4,18"

# orf3:    1+  +  +3           7+  +  +   +  +  +   +  +  +15
# orf2:              4+  +  +   +  +  +   +  +  +   +  +  +15
# orf1:    1+  +  +   +  +  +   +  +  +   +  +  +   +  +  +   +  +  +18
# exons:   |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |-  -  -  |
# pos:      1  2  3   4  5  6   7  8  9   10 11 12  13 14 15  16 17 18
# chain9:  1=  =  =3           7=  =  =9          13=  =  =   =  =  =18 # compatible - deleted ORF segment
# chain10: 1=  =  =3              8=  =9          13=  =  =   =  =  =18 # frameshift(1) - deleted ORF segment
# chain11: 1=  =  =3                 9=9          13=  =  =   =  =  =18 # frameshift(2) - deleted ORF segment
# chain12: 1=  =2                    9=9          13=  =  =   =  =  =18 # compatible - deleted ORF segment
# chain13: 1=  =  =3              8=  =9             14=  =   =  =  =18 # frameshift(2) - deleted ORF segment
# chain14: 1=  =  =3                 9=9                15=   =  =  =18 # frameshift(1) - deleted ORF segment
# chain15: 1=  =2 =3              8=  =9                15=   =  =  =18 # compatible - deleted ORF segment
# chain16: 1=  =2                 8=  =9                15=   =  =  =18 # incompatible(1) - deleted ORF segment
# chain17: 1=1                       9=9                15=   =  =  =18 # compatible - deleted ORF segment
# chain18: 1=  =2              7=  =  =9                15=   =  =  =18 # compatible - deleted ORF segment
# chain19: 1=  =2              7=  =  =9             14=  =   =  =  =18 # compatible - deleted ORF segment

orf3="1,7-3,15"
orf2="4-15"
orf1="1-18"

chain9="1,7,13-3,9,18"
chain10="1,8,13-3,9,18"
chain11="1,9,13-3,9,18"
chain12="1,9,13-2,9,18"
chain13="1,8,14-3,9,18"
chain14="1,9,15-3,9,18"
chain15="1,8,15-3,9,18"
chain16="1,8,15-2,9,18"
chain17="1,9,15-1,9,18"
chain18="1,7,15-2,9,18"
chain19="1,7,14-2,9,18"